## The Data

** Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [56]:
import pandas as pd

In [57]:
data = pd.read_csv('Data/cal_housing_clean.csv')

In [58]:
data.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [59]:
data.describe()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,28.639486,2635.763081,537.898014,1425.476744,499.539680,3.870671,206855.816909
std,12.585558,2181.615252,421.247906,1132.462122,382.329753,1.899822,115395.615874
min,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,18.000000,1447.750000,295.000000,787.000000,280.000000,2.563400,119600.000000
50%,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [60]:
data.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [61]:
x_cols = ['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population','households', 'medianIncome']
data_x = data[x_cols]
data_y = data['medianHouseValue']

In [62]:
data_x.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome
0,41.0,880.0,129.0,322.0,126.0,8.3252
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014
2,52.0,1467.0,190.0,496.0,177.0,7.2574
3,52.0,1274.0,235.0,558.0,219.0,5.6431
4,52.0,1627.0,280.0,565.0,259.0,3.8462


In [63]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data_x,data_y,test_size=0.3)

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [64]:
from sklearn.preprocessing import MinMaxScaler

In [65]:
scaler_model = MinMaxScaler()

In [66]:
scaler_model.fit(x_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [67]:
x_train.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome
13432,34.0,1345.0,265.0,834.0,290.0,3.7011
14973,10.0,3817.0,943.0,2352.0,875.0,2.1362
4957,52.0,1793.0,350.0,1303.0,366.0,3.0759
2421,18.0,1425.0,280.0,753.0,266.0,3.7813
1056,17.0,5273.0,1177.0,2446.0,1199.0,1.9362


In [68]:
x_train = pd.DataFrame(data=scaler_model.transform(x_train),columns = x_train.columns,index=x_train.index)
x_test = pd.DataFrame(data=scaler_model.transform(x_test),columns = x_test.columns,index=x_test.index)

In [70]:
x_train.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome
13432,0.647059,0.034157,0.040819,0.023291,0.047368,0.220769
14973,0.176471,0.097029,0.146050,0.065837,0.143586,0.112847
4957,1.000000,0.045552,0.054012,0.036436,0.059868,0.177653
2421,0.333333,0.036192,0.043148,0.021021,0.043421,0.226300
1056,0.313725,0.134061,0.182368,0.068472,0.196875,0.099054


### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [72]:
import tensorflow as tf

In [73]:
houseage = tf.feature_column.numeric_column('housingMedianAge')
rooms = tf.feature_column.numeric_column('totalRooms')
bedrooms = tf.feature_column.numeric_column('totalBedrooms')
population = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
income = tf.feature_column.numeric_column('medianIncome')

feat_cols = [houseage, rooms, bedrooms, population,households, income]

In [80]:
input_func = tf.estimator.inputs.pandas_input_fn(x=x_train,y=y_train,batch_size=10,num_epochs=1000,shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [89]:
dnn_model = tf.estimator.DNNRegressor(hidden_units=[6,6,6],feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\hosni\\AppData\\Local\\Temp\\tmpjlsk1oxb', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002B7AE8B1240>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [91]:
dnn_model.train(input_fn=input_func,steps=100000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\hosni\AppData\Local\Temp\tmpjlsk1oxb\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 10001 into C:\Users\hosni\AppData\Local\Temp\tmpjlsk1oxb\model.ckpt.
INFO:tensorflow:loss = 99730830000.0, step = 10001
INFO:tensorflow:global_step/sec: 530.487
INFO:tensorflow:loss = 173569840000.0, step = 10101 (0.193 sec)
INFO:tensorflow:global_step/sec: 726.627
INFO:tensorflow:loss = 71001104000.0, step = 10201 (0.137 sec)
INFO:tensorflow:global_step/sec: 701.173
INFO:tensorflow:loss = 153884490000.0, step = 10301 (0.144 sec)
INFO:tensorflow:global_step/sec: 696.296
INFO:tensorflow:loss = 157311580000.0, step = 10401 (0.144 sec)
INFO:tensorflow:global_step/sec: 686.732
INFO:tensorflow:loss = 55296664000.0, s

INFO:tensorflow:global_step/sec: 586.368
INFO:tensorflow:loss = 97169056000.0, step = 17501 (0.172 sec)
INFO:tensorflow:global_step/sec: 642.865
INFO:tensorflow:loss = 90974536000.0, step = 17601 (0.156 sec)
INFO:tensorflow:global_step/sec: 630.501
INFO:tensorflow:loss = 121053950000.0, step = 17701 (0.159 sec)
INFO:tensorflow:global_step/sec: 682.215
INFO:tensorflow:loss = 127623730000.0, step = 17801 (0.146 sec)
INFO:tensorflow:global_step/sec: 701.032
INFO:tensorflow:loss = 97959690000.0, step = 17901 (0.143 sec)
INFO:tensorflow:global_step/sec: 642.849
INFO:tensorflow:loss = 85244330000.0, step = 18001 (0.157 sec)
INFO:tensorflow:global_step/sec: 576.263
INFO:tensorflow:loss = 76160670000.0, step = 18101 (0.174 sec)
INFO:tensorflow:global_step/sec: 668.315
INFO:tensorflow:loss = 37981983000.0, step = 18201 (0.149 sec)
INFO:tensorflow:global_step/sec: 547.916
INFO:tensorflow:loss = 39763812000.0, step = 18301 (0.183 sec)
INFO:tensorflow:global_step/sec: 547.987
INFO:tensorflow:loss 

INFO:tensorflow:global_step/sec: 686.733
INFO:tensorflow:loss = 100538410000.0, step = 25401 (0.146 sec)
INFO:tensorflow:global_step/sec: 668.607
INFO:tensorflow:loss = 84422790000.0, step = 25501 (0.148 sec)
INFO:tensorflow:global_step/sec: 716.199
INFO:tensorflow:loss = 34691527000.0, step = 25601 (0.143 sec)
INFO:tensorflow:global_step/sec: 706.107
INFO:tensorflow:loss = 76785240000.0, step = 25701 (0.140 sec)
INFO:tensorflow:global_step/sec: 721.352
INFO:tensorflow:loss = 88209560000.0, step = 25801 (0.140 sec)
INFO:tensorflow:global_step/sec: 711.107
INFO:tensorflow:loss = 130671910000.0, step = 25901 (0.142 sec)
INFO:tensorflow:global_step/sec: 701.023
INFO:tensorflow:loss = 129191180000.0, step = 26001 (0.141 sec)
INFO:tensorflow:global_step/sec: 726.547
INFO:tensorflow:loss = 50183434000.0, step = 26101 (0.138 sec)
INFO:tensorflow:global_step/sec: 682.103
INFO:tensorflow:loss = 59461910000.0, step = 26201 (0.147 sec)
INFO:tensorflow:global_step/sec: 672.97
INFO:tensorflow:loss 

INFO:tensorflow:global_step/sec: 721.345
INFO:tensorflow:loss = 106953196000.0, step = 33301 (0.138 sec)
INFO:tensorflow:global_step/sec: 630.488
INFO:tensorflow:loss = 91318870000.0, step = 33401 (0.156 sec)
INFO:tensorflow:global_step/sec: 737.435
INFO:tensorflow:loss = 58074060000.0, step = 33501 (0.138 sec)
INFO:tensorflow:global_step/sec: 696.303
INFO:tensorflow:loss = 63134216000.0, step = 33601 (0.145 sec)
INFO:tensorflow:global_step/sec: 691.499
INFO:tensorflow:loss = 99963910000.0, step = 33701 (0.147 sec)
INFO:tensorflow:global_step/sec: 737.117
INFO:tensorflow:loss = 51856134000.0, step = 33801 (0.135 sec)
INFO:tensorflow:global_step/sec: 716.333
INFO:tensorflow:loss = 121709430000.0, step = 33901 (0.140 sec)
INFO:tensorflow:global_step/sec: 672.942
INFO:tensorflow:loss = 59165254000.0, step = 34001 (0.148 sec)
INFO:tensorflow:global_step/sec: 731.78
INFO:tensorflow:loss = 96046500000.0, step = 34101 (0.137 sec)
INFO:tensorflow:global_step/sec: 721.439
INFO:tensorflow:loss =

INFO:tensorflow:global_step/sec: 686.637
INFO:tensorflow:loss = 106541440000.0, step = 41201 (0.150 sec)
INFO:tensorflow:global_step/sec: 662.073
INFO:tensorflow:loss = 54238822000.0, step = 41301 (0.146 sec)
INFO:tensorflow:global_step/sec: 688.957
INFO:tensorflow:loss = 29222373000.0, step = 41401 (0.146 sec)
INFO:tensorflow:global_step/sec: 691.49
INFO:tensorflow:loss = 130450080000.0, step = 41501 (0.145 sec)
INFO:tensorflow:global_step/sec: 672.934
INFO:tensorflow:loss = 53369328000.0, step = 41601 (0.149 sec)
INFO:tensorflow:global_step/sec: 646.861
INFO:tensorflow:loss = 43277060000.0, step = 41701 (0.155 sec)
INFO:tensorflow:global_step/sec: 682.114
INFO:tensorflow:loss = 113094820000.0, step = 41801 (0.147 sec)
INFO:tensorflow:global_step/sec: 732.044
INFO:tensorflow:loss = 98848200000.0, step = 41901 (0.140 sec)
INFO:tensorflow:global_step/sec: 638.53
INFO:tensorflow:loss = 70359820000.0, step = 42001 (0.154 sec)
INFO:tensorflow:global_step/sec: 691.502
INFO:tensorflow:loss =

...

INFO:tensorflow:global_step/sec: 721.346
INFO:tensorflow:loss = 55586456000.0, step = 72801 (0.139 sec)
INFO:tensorflow:global_step/sec: 716.341
INFO:tensorflow:loss = 118984130000.0, step = 72901 (0.143 sec)
INFO:tensorflow:global_step/sec: 630.615
INFO:tensorflow:loss = 61086745000.0, step = 73001 (0.156 sec)
INFO:tensorflow:global_step/sec: 721.205
INFO:tensorflow:loss = 149310590000.0, step = 73101 (0.141 sec)
INFO:tensorflow:global_step/sec: 668.551
INFO:tensorflow:loss = 28947511000.0, step = 73201 (0.148 sec)
INFO:tensorflow:global_step/sec: 691.52
INFO:tensorflow:loss = 75421750000.0, step = 73301 (0.147 sec)
INFO:tensorflow:global_step/sec: 721.343
INFO:tensorflow:loss = 124438000000.0, step = 73401 (0.139 sec)
INFO:tensorflow:global_step/sec: 701.014
INFO:tensorflow:loss = 89147230000.0, step = 73501 (0.142 sec)
INFO:tensorflow:global_step/sec: 706.276
INFO:tensorflow:loss = 80379625000.0, step = 73601 (0.142 sec)
INFO:tensorflow:global_step/sec: 677.483
INFO:tensorflow:loss 

INFO:tensorflow:global_step/sec: 659.776
INFO:tensorflow:loss = 87415910000.0, step = 80701 (0.149 sec)
INFO:tensorflow:global_step/sec: 726.441
INFO:tensorflow:loss = 51624268000.0, step = 80801 (0.138 sec)
INFO:tensorflow:global_step/sec: 701.305
INFO:tensorflow:loss = 22431064000.0, step = 80901 (0.145 sec)
INFO:tensorflow:global_step/sec: 721.206
INFO:tensorflow:loss = 41432637000.0, step = 81001 (0.138 sec)
INFO:tensorflow:global_step/sec: 696.285
INFO:tensorflow:loss = 16877111000.0, step = 81101 (0.145 sec)
INFO:tensorflow:global_step/sec: 716.36
INFO:tensorflow:loss = 82774560000.0, step = 81201 (0.140 sec)
INFO:tensorflow:global_step/sec: 651.087
INFO:tensorflow:loss = 40397898000.0, step = 81301 (0.155 sec)
INFO:tensorflow:global_step/sec: 651.083
INFO:tensorflow:loss = 101278600000.0, step = 81401 (0.153 sec)
INFO:tensorflow:global_step/sec: 642.62
INFO:tensorflow:loss = 42762707000.0, step = 81501 (0.155 sec)
INFO:tensorflow:global_step/sec: 659.778
INFO:tensorflow:loss = 1

INFO:tensorflow:global_step/sec: 691.472
INFO:tensorflow:loss = 108335610000.0, step = 88601 (0.144 sec)
INFO:tensorflow:global_step/sec: 742.908
INFO:tensorflow:loss = 143522700000.0, step = 88701 (0.135 sec)
INFO:tensorflow:global_step/sec: 742.574
INFO:tensorflow:loss = 90865420000.0, step = 88801 (0.137 sec)
INFO:tensorflow:global_step/sec: 691.628
INFO:tensorflow:loss = 42805813000.0, step = 88901 (0.145 sec)
INFO:tensorflow:global_step/sec: 726.576
INFO:tensorflow:loss = 100635650000.0, step = 89001 (0.137 sec)
INFO:tensorflow:global_step/sec: 726.886
INFO:tensorflow:loss = 23888206000.0, step = 89101 (0.139 sec)
INFO:tensorflow:global_step/sec: 668.451
INFO:tensorflow:loss = 138686350000.0, step = 89201 (0.147 sec)
INFO:tensorflow:global_step/sec: 711.122
INFO:tensorflow:loss = 57086360000.0, step = 89301 (0.142 sec)
INFO:tensorflow:global_step/sec: 753.73
INFO:tensorflow:loss = 38291660000.0, step = 89401 (0.134 sec)
INFO:tensorflow:global_step/sec: 696.437
INFO:tensorflow:loss

INFO:tensorflow:global_step/sec: 675.761
INFO:tensorflow:loss = 108460196000.0, step = 96501 (0.150 sec)
INFO:tensorflow:global_step/sec: 711.115
INFO:tensorflow:loss = 26814185000.0, step = 96601 (0.140 sec)
INFO:tensorflow:global_step/sec: 701.175
INFO:tensorflow:loss = 44393920000.0, step = 96701 (0.144 sec)
INFO:tensorflow:global_step/sec: 711.112
INFO:tensorflow:loss = 68182753000.0, step = 96801 (0.142 sec)
INFO:tensorflow:global_step/sec: 682.085
INFO:tensorflow:loss = 21116290000.0, step = 96901 (0.143 sec)
INFO:tensorflow:global_step/sec: 672.811
INFO:tensorflow:loss = 40443167000.0, step = 97001 (0.150 sec)
INFO:tensorflow:global_step/sec: 753.864
INFO:tensorflow:loss = 78431170000.0, step = 97101 (0.134 sec)
INFO:tensorflow:global_step/sec: 737.444
INFO:tensorflow:loss = 43063812000.0, step = 97201 (0.135 sec)
INFO:tensorflow:global_step/sec: 705.976
INFO:tensorflow:loss = 75690230000.0, step = 97301 (0.144 sec)
INFO:tensorflow:global_step/sec: 711.258
INFO:tensorflow:loss =

INFO:tensorflow:loss = 110087390000.0, step = 104301 (0.150 sec)
INFO:tensorflow:global_step/sec: 721.341
INFO:tensorflow:loss = 85511650000.0, step = 104401 (0.137 sec)
INFO:tensorflow:global_step/sec: 726.411
INFO:tensorflow:loss = 96665035000.0, step = 104501 (0.141 sec)
INFO:tensorflow:global_step/sec: 716.225
INFO:tensorflow:loss = 83223940000.0, step = 104601 (0.138 sec)
INFO:tensorflow:global_step/sec: 634.708
INFO:tensorflow:loss = 51939426000.0, step = 104701 (0.159 sec)
INFO:tensorflow:global_step/sec: 646.891
INFO:tensorflow:loss = 70291590000.0, step = 104801 (0.155 sec)
INFO:tensorflow:global_step/sec: 672.931
INFO:tensorflow:loss = 39416586000.0, step = 104901 (0.147 sec)
INFO:tensorflow:global_step/sec: 737.096
INFO:tensorflow:loss = 87104225000.0, step = 105001 (0.139 sec)
INFO:tensorflow:global_step/sec: 721.375
INFO:tensorflow:loss = 41528144000.0, step = 105101 (0.139 sec)
INFO:tensorflow:global_step/sec: 706.626
INFO:tensorflow:loss = 40403640000.0, step = 105201 (0

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [95]:
input_fn_predict = tf.estimator.inputs.pandas_input_fn(x=x_test,batch_size=10,num_epochs=1,shuffle=False)

In [96]:
ll = dnn_model.predict(input_fn=input_fn_predict)

In [97]:
list(ll)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\hosni\AppData\Local\Temp\tmpjlsk1oxb\model.ckpt-110000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[{'predictions': array([241030.27], dtype=float32)},
 {'predictions': array([279411.34], dtype=float32)},
 {'predictions': array([192075.08], dtype=float32)},
 {'predictions': array([150273.67], dtype=float32)},
 {'predictions': array([206336.75], dtype=float32)},
 {'predictions': array([182055.69], dtype=float32)},
 {'predictions': array([200304.73], dtype=float32)},
 {'predictions': array([409282.53], dtype=float32)},
 {'predictions': array([166420.8], dtype=float32)},
 {'predictions': array([229907.14], dtype=float32)},
 {'predictions': array([134304.27], dtype=float32)},
 {'predictions': array([209335.02], dtype=float32)},
 {'predictions': array([232012.19], dtype=float32)},
 {'predictions': array([284400.5], dtype=float32)},
 {'predictions': array([161151.2], dtype=float32)},
 {'predictions': array([208878.8], dtype=float32)},
 {'predictions': array([143753.11], dtype=float32)},
 {'predictions': array([203704.77], dtype=float32)},
 {'predictions': array([200005.5], dtype=float32)}

** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [114]:
preds = []
for pred in dnn_model.predict(input_fn=input_fn_predict):
    preds.append(pred['predictions'])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\hosni\AppData\Local\Temp\tmpjlsk1oxb\model.ckpt-110000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [104]:
from sklearn.metrics import mean_squared_error

In [118]:
print(len(preds))
y_test.shape

6192


(6192,)

In [119]:
mean_squared_error(y_test,preds)**0.5

86200.10926577222